# Preprocee input data

In [1]:
import os, sys
import pandas as pd

# import util functions from local package 'util'
package_dir = os.path.abspath(f"./util")
if package_dir not in sys.path:
    sys.path.insert(0, package_dir)
from util import remove_irrelevant_text

df = pd.read_csv('./data/ECG_Absert.csv')

# remove misleading but current trace irrelevant heading and ending.
# replace '\n' with '   '
# remove space in start and end 
df.rename(columns={'text':'text_original'},inplace=True)
df['text'] = df['text_original'].apply(lambda x: remove_irrelevant_text(x).replace('\n', '   ').strip())

df = df[df['text']!='']

df.to_csv('./data/ECG_data.csv', index=False)


# Launch EnsemLLMs 

## Run EnsemLLMs end-to-end
(1) Init: load configuration for EnsemLLMs

(2) Launch: run each LLM agent for the task

(3) Aggregate: aggregate the results from each LLM agent 

(4) Vote: Vote for final result

In [1]:
# note:
# skip this method, because later var_val_list configuration will be changed,
# as we infer new AF values from LLMs' output values.

# %run setup.py    
# from ensemble_llms_app import run

# # launch the ensemble project and run each specified LLM agent.
# run()


## Launch EnsemLLMs step-by-step

(1) Init: load configuration for EnsemLLMs 

(2) Launch: run each LLM agent for the task

(3) Infer new values of AF_Diagnosis from each LLM's output

(4) Aggregate: aggregate the results from each LLM agent 

(5) Vote: Vote for final result

In [1]:
import os
import pandas as pd
import json

%run setup.py    
from ensemble_llms_app import run


In [2]:
# Load configuration of the ensemble project.
ensemLLMs = run('init')


Home path: /project/DPDS/Xiao_lab/s221963/2025_LLMsAgents/demo


In [4]:
# launch the ensemble project and run each specified LLM agent.
df = ensemLLMs.launch()


Start at: 2025-08-28 15:12:22.109843
LLM agent app home path: /project/DPDS/Xiao_lab/s221963/2025_LLMsAgents/demo/beluga-70b_S02
Model: beluga-70b
Dataset: ECG
Start at: 2025-08-28 15:12:22.233112
0:  R200199  
End at: 2025-08-28 15:12:44.169232
Elapsed time: 0.365602 minutes (21.936120 seconds).

Failed requests: []
Failed json loading: 0
[]
Input Tokens: 1291
Output Tokens: 119
Total Tokens: 1410
Model stable-beluga:70b is not in the cost list.

The output csv file saved to /project/DPDS/Xiao_lab/s221963/2025_LLMsAgents/demo/beluga-70b_S02/data/ECG_beluga-70b_S02.csv.
Computing with beluga-70b for ECG succeed.

LLM agent app home path: /project/DPDS/Xiao_lab/s221963/2025_LLMsAgents/demo/gemma7b_S02
Model: gemma7b
Dataset: ECG
Start at: 2025-08-28 15:12:44.293035
0:  R200199  
End at: 2025-08-28 15:12:49.325113
Elapsed time: 0.083868 minutes (5.032079 seconds).

Failed requests: []
Failed json loading: 0
[]
Input Tokens: 1068
Output Tokens: 89
Total Tokens: 1157
Model gemma:7b is not 

In [4]:
#

In [5]:
# postprocessing

for app in ensemLLMs.llm_apps_specs:
    predictor_home_path = os.path.join(ensemLLMs.home_path, f"{app['model_id']}_{ensemLLMs.prompt_id}")
    output_data_filename = f"{ensemLLMs.dataset_id}_{app['model_id']}_{ensemLLMs.prompt_id}"
    output_data_path = os.path.join(predictor_home_path,f"data/{output_data_filename}.csv")
    output_rawdata_path = os.path.join(predictor_home_path,f"data/{output_data_filename}_raw.csv")
    # print(output_data_path)
    
    # infer new categorical value from old one in var 'AF_Diagnosis' for each LLM output ---------------------------
    
    # rename original llm output data file by adding '_raw' in the end
    if not os.path.exists(output_rawdata_path):
        os.rename(output_data_path,output_rawdata_path)
        # shutil.copy(src,dst)
    
    df = pd.read_csv(output_rawdata_path,keep_default_na=False)

    # keep original value in col 'AF_Diagnosis_raw'
    df.rename(columns={'AF_Diagnosis':'AF_Diagnosis_raw'},inplace=True)
    df['AF_Diagnosis'] = ''

    # infer new value
    df['AF_Diagnosis'] = df['AF_Diagnosis_raw'].apply(lambda x: 'AF' if x in ['AF', 'Probable AF', 'AFib', 'AFL'] 
                                                                   else 'Uncertain' if x in ['Possible AF', 'Not Specified']
                                                                   else 'Non-AF' if x in ['Non-AF']
                                                                   else 'NA')
    # the new values for 'AF_Diagnosis' makes original ensemLLMs' config of var_val_list need changing to new var_val_list
    var_val_dict = {"AF_Diagnosis": ["AF","Uncertain","Non-AF"]}
    
    df.to_csv(output_data_path, index=False)
    

In [6]:
df = ensemLLMs.aggregate()

# use the new var_val_list for configuration
ensemLLMs.vote(df, revised_var_val_dict=var_val_dict)



Read in prediction from /project/DPDS/Xiao_lab/s221963/2025_LLMsAgents/demo/beluga-70b_S02/data/

Read in prediction from /project/DPDS/Xiao_lab/s221963/2025_LLMsAgents/demo/gemma7b_S02/data/

Read in prediction from /project/DPDS/Xiao_lab/s221963/2025_LLMsAgents/demo/llama3-70b_S02/data/

Read in prediction from /project/DPDS/Xiao_lab/s221963/2025_LLMsAgents/demo/mistral-openorca_S02/data/

Read in prediction from /project/DPDS/Xiao_lab/s221963/2025_LLMsAgents/demo/openhermes_S02/data/

Read in prediction from /project/DPDS/Xiao_lab/s221963/2025_LLMsAgents/demo/qwen-72b_S02/data/

Read in prediction from /project/DPDS/Xiao_lab/s221963/2025_LLMsAgents/demo/qwen2-72b_S02/data/

Save the aggregated data into data/ECG_Free7Demo_aggregated.csv
Voting by predictors: ['beluga-70b_S02', 'gemma7b_S02', 'llama3-70b_S02', 'mistral-openorca_S02', 'openhermes_S02', 'qwen-72b_S02', 'qwen2-72b_S02']
Vote result saved to data/ECG_Free7Demo_voted.csv.


,note_id,text_original,AF_gt,text,AF_Diagnosis_beluga-70b_S02,AF_pr_beluga-70b_S02,Explanation_beluga-70b_S02,AF_Diagnosis_gemma7b_S02,AF_pr_gemma7b_S02,Explanation_gemma7b_S02,...,Explanation_qwen-72b_S02,AF_Diagnosis_qwen2-72b_S02,AF_pr_qwen2-72b_S02,Explanation_qwen2-72b_S02,votes_AF_Diagnosis_AF,votes_AF_Diagnosis_Uncertain,votes_AF_Diagnosis_Non-AF,votes_AF_Diagnosis_non-standard,votes_AF_Diagnosis_no_response,AF_Diagnosis_voted
0,R200199,Clinical indication for EKG: Z79.899 - Other...,AF,Baseline artifact. Probable atrial flutter wi...,AF,0.95,The report states 'Probable atrial flutter wit...,AF,0.85,The ECG report states 'probable atrial flutter...,...,The ECG report identifies the cardiac rhythm a...,AF,0.95,"The report states 'Probable atrial flutter', w...",5,0,1,1,0,AF
